# AIの5ステップ（学習するとこから）
### (1) 学習(訓練)データとテストデータの用意
### (2) モデルの構築とハイパーパラメータの設定
### (3) モデルの学習（エポックの繰り返し）
### (4) モデルの評価（テスト）、テスト結果のグラフ化
### (5) モデル（プログラム）の保存、学習済みモデル（オブジェクト）の保存

# AIの3ステップ（学習済みを使う PreTrained-Model）
### (1) テストデータの用意
### (2) 構築済みモデルのロード
### (3) モデルの評価（テスト）、テスト結果のグラフ化

# (1) 学習(訓練)データとテストデータの用意(MNISTの場合)

In [31]:
from torch import utils
from torchvision import datasets
import torchvision.transforms as transforms
import torch
from torch import nn, optim
from torch.nn import functional as F

# 学習データの用意
trainset = datasets.MNIST(root='./data', train=True, download=True, transform=transforms.ToTensor())
# ミニバッチごとにデータを纏める(学習時にはshuffle=True)
train_loader = utils.data.DataLoader(trainset, batch_size=100, shuffle=True, num_workers=2)
# テストデータの用意
testset = datasets.MNIST(root='./data', train=False, download=True, transform=transforms.ToTensor())
# ミニバッチごとにデータを纏める(テスト時にはshuffle=False)
test_loader = utils.data.DataLoader(testset, batch_size=100, shuffle=False, num_workers=2)

In [40]:
# データの確認
# trainset[0]は0番目のデータ
# trainset[0][0]は0番目のデータの画像データ
# train_loaderはミニバッチごとにデータを纏めたもの
# train_loader.datasetはtrainsetと同じ
# train_loader.dataset[0]は0番目のデータ
# train_loader.dataset[0][0]は0番目のデータの画像データ
# train_loader.dataset[0][1]は0番目のデータのラベル

print(trainset[0][0]) # は0番目のデータの画像データ
print(train_loader)   # はミニバッチごとにデータを纏めたもの
print(train_loader.dataset)  # はtrainsetと同じ
# train_loader.dataset[0]は0番目のデータ
# train_loader.dataset[0][0]は0番目のデータの画像データ
print(train_loader.dataset[0][1])  # は0番目のデータのラベル

tensor([[[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,

# (2) モデルの構築

In [23]:
# GPUの設定
if torch.backends.mps.is_available():
    device = torch.device('mps')
else:
    print("MPS is not available")
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# モデルの構築
class mlp_net(nn.Module):
    # モデルの構築、初期化、設定
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(784, 512)
        self.fc2 = nn.Linear(512, 10)
        self.criterion = nn.MSELoss()
        self.optimizer = optim.Adam(self.parameters())
    # 順伝播
    def forward(self, x):
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        return x

MPS is available


# (3) モデルの学習

In [28]:
# モデルの学習
def train(model, train_loader):
    # 今は学習時であることを明示するコード
    model.train()
    # ミニバッチごとにループさせる,train_loaderの中身を出し切ったら1エポックとなる
    for batch_imgs, batch_labels in train_loader:
        batch_imgs = batch_imgs.reshape(-1, 28*28*1).to(device)  # 画像データを1次元に変換
        labels = torch.eye(10)[batch_labels].to(device)  # 正解ラベルをone-hotベクトルへ変換

        outputs = model(batch_imgs)  # 順伝播
        model.optimizer.zero_grad()  # 勾配を初期化（前回のループ時の勾配を削除）
        loss = model.criterion(outputs, labels)  # 損失を計算
        loss.backward()  # 逆伝播で勾配を計算
        model.optimizer.step()  # 最適化
    return

In [29]:
#
def train2(model, train_loader):
    # 今は学習時であることを明示するコード
    model.train()

    ### 追記部分1 ###
    # 正しい予測数、損失の合計、全体のデータ数を数えるカウンターの0初期化
    total_correct = 0
    total_loss = 0
    total_data_len = 0
    ### ###

    # ミニバッチごとにループさせる,train_loaderの中身を出し切ったら1エポックとなる
    for batch_imgs, batch_labels in train_loader:

        batch_imgs = batch_imgs.reshape(-1, 28*28*1).to(device)   # 画像データを1次元に変換
        labels = torch.eye(10)[batch_labels].to(device)   # 正解ラベルをone-hotベクトルへ変換
        # plt.imshow(batch_imgs.cpu())
        # plt.show()

        outputs = model(batch_imgs)  # 順伝播
        model.optimizer.zero_grad()  # 勾配を初期化（前回のループ時の勾配を削除）
        loss = model.criterion(outputs, labels)  # 損失を計算
        loss.backward()  # 逆伝播で勾配を計算
        model.optimizer.step()  # 最適化

        ### 追記部分2 ###
        # ミニバッチごとの正答率と損失を求める
        _, pred_labels = torch.max(outputs, axis=1)  # outputsから必要な情報(予測したラベル)のみを取り出す。
        batch_size = len(batch_labels)  # バッチサイズの確認
        for i in range(batch_size):  # データ一つずつループ,ミニバッチの中身出しきるまで
            total_data_len += 1  # 全データ数を集計
            if pred_labels[i] == batch_labels[i]:
                total_correct += 1 # 正解のデータ数を集計
        total_loss += loss.item()  # 全損失の合計

    # 今回のエポックの正答率と損失を求める
    accuracy = total_correct/total_data_len*100  # 予測精度の算出
    loss = total_loss/total_data_len  # 損失の平均の算出
    return accuracy, loss

In [30]:
# モデルを宣言する
model = mlp_net().to(device)

# 学習させ、その結果を表示する
acc, loss = train(model, train_loader)
print(f'正答率: {acc}, 損失: {loss}')

TypeError: cannot unpack non-iterable NoneType object